In [1]:
import os
from openai import AzureOpenAI, OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = AzureOpenAI(
    api_version="2025-01-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_KEY"),
)

# client = OpenAI(
#     api_key=os.getenv("OPENAI_API_KEY")
# )

In [3]:
def chat(messages):
    response = client.chat.completions.create(
        messages=messages,
        max_tokens=16384,
        temperature=1.0,
        top_p=1.0,
        model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    )
    return response.choices[0].message.content

In [4]:
def chat_with_history(history, user_input):
    messages = history + [{
        "role": "user",
        "content": user_input,
    }]
    response = chat(messages)
    messages.append({
        "role": "assistant",
        "content": response,
    })
    return messages, response

In [5]:
history = [
    {"role": "system", "content": "You are a helpful assistant that tells jokes in poem."},
]

history, response = chat_with_history(history, "tell me a joke")
history, response = chat_with_history(history, "tell me a another joke")

In [6]:
history, response = chat_with_history(history, "tell me a third joke")
print(response)

Sure thing — here’s your third joke in poem form:  

---

A chicken once signed up for school,  
Hoping to learn every rule.  
It studied real hard,  
And became a barnyard guard…  
Now it yells, “No crossing without a cluck-pass, fool!”  

---

Do you want me to make a *really* funny one with a twist ending next?


In [20]:
import requests
def get_weather(location):
    response = requests.get(f"http://wttr.in/{location}?format=3")
    return response.text

In [21]:
get_weather("Bangalore")

'Bangalore: ⛅️  +27°C\n'

In [31]:
system_prompt = """
You are a helpful assistant.
You have access to following tools:
1. get_weather(location): A function that returns current weather in location.
2. foo(): A function that does nothing.

You must use the tools to answer user questions. For using the tool you must use the following format:
User: <user question>
Thought: <your thought process>
Tool: <tool name>
Input: <input to the tool>
Observation: <output of the tool. Output from the tool will be passed in subsequent calls.>

If you do not need to use a tool any more, you can answer the question directly.
"""

history = [
    {"role": "system", "content": system_prompt},
]
history, response = chat_with_history(history, "What activities should I do in Bangalore today?")
print(response)


Thought: To recommend suitable activities, it would help to know the current weather in Bangalore, as it may influence whether outdoor or indoor plans are better.  
Tool: get_weather(location)  
Input: Bangalore  


In [ ]:
if "get_weather" in response:
    location = response.split("Input:")[-1].strip()
    current_weather = get_weather(location)
    history, response = chat_with_history(history, f"Observation: {current_weather}")
print(response)

Since Melbourne is currently cloudy with a cool temperature of 11°C, outdoor activities that don’t rely on sunny weather are ideal, along with some indoor options to keep warm.

Here are some suggestions for today:  
1. **Explore the National Gallery of Victoria (NGV)** – Perfect for a quiet day appreciating art indoors.  
2. **Visit Melbourne Museum** – Learn about history, science, and culture while staying warm.  
3. **Stroll through the Royal Botanic Gardens** – The weather is good for an invigorating walk without worrying about harsh sun.  
4. **Take a coffee crawl through Melbourne’s laneways** – The city’s café culture is unmatched, and cloudy weather makes it extra cozy.  
5. **Check out Queen Victoria Market** – Great for picking up fresh produce and unique goods.  
6. **Attend a theatre show or live music event** – Melbourne has a vibrant arts scene worth exploring.  

If you’d like, I can also suggest **specific events happening in Melbourne today**. Would you like me to do 

In [18]:
print(history)

[{'role': 'system', 'content': '\nYou are a helpful assistant.\nYou have access to following tools:\n1. get_weather(): A function that returns current weather in melbourne.\n2. foo(): A function that does nothing.\n\nYou must use the tools to answer user questions. For using the tool you must use the following format:\nUser: <user question>\nThought: <your thought process>\nTool: <tool name>\nInput: <input to the tool>\nObservation: <output of the tool. Output from the tool will be passed in subsequent calls.>\n\nIf you do not need to use a tool any more, you can answer the question directly.\n'}, {'role': 'user', 'content': 'What is weather in melbourne?'}, {'role': 'assistant', 'content': 'Thought: The user is asking about the current weather in Melbourne, so I should use the get_weather() tool.  \nTool: get_weather()  \nInput: None  '}, {'role': 'user', 'content': 'Observation: Melbourne: ☁️   +11°C\n'}, {'role': 'assistant', 'content': 'The current weather in Melbourne is cloudy wi